In [1]:
!pip install milvus milvus[model] pymilvus pymilvus[model]
!pip install sentence_transformers datasets accelerate
!pip install torch transformers

# !rm -rf /kaggle/working/ragbench.db

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 MB 30.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 39.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 80.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.1 MB/s eta 0:00:0000:01


In [2]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("BAAI/LLM-Embedder")
#tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Initialize the model
embedder = SentenceTransformer("BAAI/LLM-Embedder") 
# embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2") 


tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/28.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
# New code - 12/4 10 pm

from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer

# Sliding window configuration
TOKEN_LIMIT = 512
SLIDING_WINDOW_OVERLAP = 100  # Overlap between consecutive chunks (in tokens)

# Function for chunking with token limit and sliding window
def chunk_with_token_limit(text, token_limit, overlap):
    sentences = sent_tokenize(text)  # Split text into sentences
    chunks = []  # Store resulting chunks
    current_chunk = []  # Temporarily hold sentences for the current chunk
    current_chunk_tokens = 0  # Token count for the current chunk

    for sentence in sentences:
        # Tokenize the sentence and calculate its token count
        sentence_tokens = tokenizer.tokenize(sentence)
        num_tokens = len(sentence_tokens)

        # print(f"Tokens: {sentence_tokens[0]}")

        # If adding this sentence exceeds the token limit
        if current_chunk_tokens + num_tokens > token_limit:
            # Save the current chunk
            chunk_text = " ".join(current_chunk)
            chunks.append(chunk_text)

            # Prepare the next chunk with overlap
            overlap_tokens = tokenizer.tokenize(" ".join(current_chunk[-1:]))
            current_chunk = [sentence for sentence in current_chunk[-(overlap // len(overlap_tokens)) :]] if current_chunk else []
            current_chunk_tokens = sum(len(tokenizer.tokenize(sent)) for sent in current_chunk)

        # Add the sentence to the current chunk
        current_chunk.append(sentence)
        current_chunk_tokens += num_tokens

    # Add the last chunk if it exists
    if current_chunk:
        chunk_text = " ".join(current_chunk)
        chunks.append(chunk_text)

    return chunks

def process_document_with_identifiers(document):
    processed_data = []
    title_count = -1  # to start from 0
    # print("document>>>>>>>",document)
    for section in document:
        section_chunks = []
        passage_count = [ord('a')]  # Passage identifier as a list to handle nested increments
        title_count += 1  # Increment title count

        # Tokenize the section into sentences
        sentences = sent_tokenize(section)
        for sentence in sentences:
            if sentence.startswith("Title:"):
                # New document detected
                identifier = f"{title_count}{''.join(chr(c) for c in passage_count)}"  # Identifier for the title
                chunked_texts = chunk_with_token_limit(sentence, TOKEN_LIMIT, SLIDING_WINDOW_OVERLAP)
                for chunk in chunked_texts:
                    section_chunks.append([identifier, chunk])
                passage_count = [ord('a')]  # Reset passage count for the new document
            else:
                # Sentence under the current document
                identifier = f"{title_count}{''.join(chr(c) for c in passage_count)}"
                chunked_texts = chunk_with_token_limit(sentence, TOKEN_LIMIT, SLIDING_WINDOW_OVERLAP)
                #print("chunked_texts>>>>process_document_with_identifiers>>>>> "+ "".join(chunked_texts))
                for chunk in chunked_texts:
                    section_chunks.append([identifier, chunk])

                # Increment passage_count intelligently
                i = len(passage_count) - 1
                while i >= 0:
                    passage_count[i] += 1
                    if passage_count[i] > ord('z'):
                        passage_count[i] = ord('a')
                        if i == 0:
                            passage_count.insert(0, ord('a'))  # Add a new character to the identifier
                        i -= 1
                    else:
                        break


        # print("section_chunks>>>>>>>",section_chunks)
        processed_data.append(section_chunks)

    return processed_data

### **Class to handle data in Milvus** ###

In [5]:
import numpy as np
from pymilvus import connections
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection
from pymilvus import MilvusClient
from pymilvus import utility

class VectorDataStore:
    db_url = "http://localhost:19530"

    #description = f"collection created for {self.name}"

    def __init__(self, path="/content/ragbench.db"):
        self.client = MilvusClient(path)


    def get_or_create_collection(self, name, vec_dim=128):
        try:
            self.get_collection(name)
        except:
            print(f"Collection {name} doesn't exist. Creating...")
            self.create_collection(name, vec_dim)
        

    def create_collection(self, name, vec_dim=128):
        if self.client.has_collection(name):
            self.default_collection_name = name

        self.description = f"collection to store {name}"

        index_params = self.client.prepare_index_params()
        index_params.add_index(
            field_name="embedding",
            index_type="AUTOINDEX",
            metric_type="COSINE"
        )
        schema = self.client.create_schema(
            auto_id=False,
            enable_dynamic_fields=True,
        )
        schema.add_field(field_name="pk", datatype=DataType.VARCHAR, max_length=64, is_primary=True)
        schema.add_field(field_name="metadata", datatype=DataType.JSON)
        schema.add_field(field_name="documents", datatype=DataType.VARCHAR, max_length=512)
        schema.add_field(field_name="embedding", datatype=DataType.FLOAT_VECTOR, dim=vec_dim)

        collection = self.client.create_collection(collection_name=name,
                                       schema=schema,
                                       index_params=index_params)
        self.current_collection = collection
        return collection


    def get_collection(self, name):
        if not self.client.has_collection(name):
            raise ValueError(f"Collection '{name}' does not exist.")
        self.current_collection = Collection(name)
        return self.current_collection

    def get_all_records(self, collection):
        all_records = self.client.query(
            collection_name=collection,
            filter=None,
            output_fields=["documents", "metadata"],
            limit=10000
        )
        if all_records == None:
            all_records = []

        return all_records

    def has_entities(self, name):
        if not self.client.has_collection(name):
            raise ValueError(f"Collection '{name}' does not exists.")
        self.default_collection = name
        collection_stats = self.client.get_collection_stats(collection_name)
        count = collection_stats.get("row_count", 0)  # Retrieve the number of entities
        return count

    def insert(self, collection_name: str, metadata: list[dict[str, any]],
                documents: list[str], embeddings: np.ndarray, ids: list[int]):

        if not self.client.has_collection(collection_name):
            raise ValueError(f"Collection '{collection_name}' does not exist. Create it first.")

        if len(metadata) != len(embeddings) != len(documents) != len(ids):
           raise ValueError("Metadata, documnets, ids and embeddings must have the same length.")

        data = []
        for meta, doc, emb, id in zip(metadata, documents, embeddings, ids):
          datum = {
              "pk": id,
              "metadata": meta,
              "documents": doc,
              "embedding": emb.tolist(),
          }
          data.append(datum)

        self.client.insert(collection_name, data)
        print(f"Inserted {len(metadata)} records into collection '{collection_name}'.")

    def drop_collection(self, collection_name: str):
        if not self.client.has_collection(collection_name):
            raise ValueError(f"Collection '{collection_name}' does not exist.")
        self.client.drop_collection(collection_name)
        print(f"Dropped collection '{collection_name}'.")

    def delete_all(self, collection_name: str):
        if not self.client.has_collection(collection_name):
            raise ValueError(f"Collection '{collection_name}' does not exist.")
        self.client.delete(collection_name, expr="pk >= 0")
        self.client.flush([collection_name])

    def search(self, query_embedding: np.ndarray, top_k: int = 10) -> list[dict[str, any]]:
        """
        Search across all collections for the top-k closest embeddings.
        :param query_embedding: The embedding vector to search for.
        :param top_k: Number of top results to retrieve.
        :return: A list of dictionaries containing collection name, id, metadata, and distance.
        """
        results = []
        #collections = self.client.list_collections()
        collections = ["ragbench_collection_techqa_v09"]
        start_time = time.time()
        for collection_name in collections:
            if not self.client.has_collection(collection_name):
                continue

            # Set params to COSINE to match chromadb
            search_params = {"metric_type": "COSINE", "params": {"ef": 128}}

            search_results = self.client.search(
                collection_name=collection_name,
                data=[query_embedding],
                anns_field="embedding",
                search_params=search_params,
                limit=top_k,
                output_fields=["metadata", "documents"]
            )

            for hits in search_results:
                for hit in hits:
                    print(f"Collection: {collection_name}, data: {str(hit)}")
                    results.append({
                        "collection": collection_name,
                        "id": hit["id"],
                        "metadata": hit["entity"]["metadata"],
                        "distance": hit["distance"],
                        "documents": hit["entity"]["documents"]
                      })

        results = sorted(results, key=lambda x: x["distance"])[:top_k]
        end_time = time.time()
        print(f"Search completed. Found {len(results)} results. in {end_time - start_time} secs")
        return results

    def extract_documents(self, search_results: list[dict[str, any]]) -> list[np.ndarray]:
      """
      Extract embedding values from search results.
      :param search_results: List of dictionaries containing search results.
      :return: List of embedding vectors as NumPy arrays.
      """
      return [np.array(result["documents"]) for result in search_results if "documents" in result]

#### **Functions to generate and retrieve hashes** ####

In [6]:
import hashlib

# Function to generate a hash based on content and key metadata
def generate_hash(content, metadata):
    """Generate a unique hash for the document content and key metadata."""
    key_fields = f"{content}|{metadata.get('item_index')}|{metadata.get('prefix')}"
    return hashlib.md5(key_fields.encode('utf-8')).hexdigest()

# Function to retrieve existing hashes from the database
def get_existing_hashes_milvus(all_records):
    """Retrieve all existing hashes (IDs) currently in the database."""
    existing_hashes = set()
    print(f"all records >>> {len(all_records)}")    
    if all_records == None or len(all_records) == 0:
        return existing_hashes
        
    for doc, metadata in zip(all_records["documents"], all_records["metadata"]):
        doc_hash = generate_hash(doc, metadata)
        existing_hashes.add(doc_hash)
    return existing_hashes

### Tokenize and generate documents ###

In [7]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

datasets = [
    'covidqa', 'cuad', 'delucionqa', 'emanual', 'expertqa',
    'finqa', 'hagrid', 'hotpotqa', 'msmarco', 'pubmedqa',
    'tatqa', 'techqa'
]

# Initialize storage for documents, IDs, and metadata
all_documents = []
all_ids = []
all_metadatas = []

# Process each dataset
doc_idx = 0  # Global document index for unique IDs
for dataset in datasets:
    data = load_dataset("rungalileo/ragbench", dataset, split="train")
    # only select first 5 records for debugging duplicate records. **PLEASE REMOVE THIS AFTER DEBUGGING**
    # data = data.select(range(5))
    for idx, row in tqdm(enumerate(data), desc=f"Processing {dataset}"):
        # Extract document text
        doc_text = row.get('documents', '')

        # Skip if no documents found
        if not doc_text:
            continue

        # Process the document
        processed_output = process_document_with_identifiers(doc_text)
        added_item_idxs = set()

        # Populate the lists
        for section_idx, section in enumerate(processed_output):
            for item_idx, (prefix, content) in enumerate(section):
                # Skip if this item_idx has already been processed
                if item_idx in added_item_idxs:
                    continue

                # Add the item_idx to the set to track it
                added_item_idxs.add(item_idx)

                # Add the document
                document = f"[{prefix}] {content}"
                all_documents.append(document)

                # Construct a globally unique ID
                doc_id = f"{dataset}_{doc_idx}_{section_idx}_{item_idx}"
                all_ids.append(doc_id)

                # Construct metadata
                metadata = {
                    "dataset": dataset,
                    "global_index": doc_idx,
                    "section_index": section_idx,
                    "item_index": item_idx,
                    "prefix": prefix,
                    "type": "Title" if prefix.endswith("a") else "Passage",
                }
                all_metadatas.append(metadata)

        doc_idx += 1  # Increment global document index

# Step 4: Generate Embeddings
batch_size = 2500  # Adjust based on available memory

# Generate embeddings in batches
all_embeddings = []
for i in tqdm(range(0, len(all_documents), batch_size), desc="Generating embeddings"):
    batch_docs = all_documents[i:i + batch_size]
    batch_embeddings = embedder.encode(batch_docs, show_progress_bar=True)
    all_embeddings.extend(batch_embeddings)

README.md:   0%|          | 0.00/24.7k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.20M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/854k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/913k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1252 [00:00<?, ? examples/s]

...

Generating embeddings: 100%|█████████▉| 400/402 [1:10:38<00:25, 12.70s/it]

Batches:   0%|          | 0/79 [00:00<?, ?it/s]

Generating embeddings: 100%|█████████▉| 401/402 [1:10:51<00:12, 12.59s/it]

Batches:   0%|          | 0/23 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 402/402 [1:10:54<00:00, 10.58s/it]


### initialize milvus and add data to db ###

In [8]:
datastor = VectorDataStore('/kaggle/working/ragbench.db')
collection_name = "ragbench_collection_all_baai_v1"
insert_data = False
store_client = "Milvus"
num_records = 0

vector_dim = embedder.get_sentence_embedding_dimension()

datastor.get_or_create_collection(collection_name, vector_dim)
num_records = datastor.has_entities(collection_name)
if num_records == 0:
    insert_data = True

print(f"count >>> {num_records} insert_data >>> {insert_data}")

Collection ragbench_collection_all_baai_v1 doesn't exist. Creating...
count >>> 0 insert_data >>> True


In [ ]:
existing_hashes = get_existing_hashes_milvus(datastor.get_all_records(collection_name))

for i in tqdm(range(0, len(all_documents), batch_size), desc="Adding data to DB"):
    batch_embeddings = all_embeddings[i:i + batch_size]
    batch_metadatas = all_metadatas[i:i + batch_size]
    batch_documents = all_documents[i:i + batch_size]
    batch_ids = []

    # Generate hashes for each document in the batch
    for doc, metadata in zip(batch_documents, batch_metadatas):
        doc_hash = generate_hash(doc, metadata)
        if doc_hash not in existing_hashes:
            batch_ids.append(doc_hash)
            existing_hashes.add(doc_hash)  # Add hash to local set to avoid duplicates in the same batch
        else:
            print(f"Skipping duplicate document: {doc[:15]}...")  # Print a preview of the duplicate doc

    # Add non-duplicate documents to the database
    if batch_ids:  # Ensure there are non-duplicate documents to add
        # Add the batch to the Milvus collection
        if store_client == "Milvus" and insert_data:
            datastor.insert(collection_name,
                metadata=batch_metadatas,
                documents=batch_documents,
                embeddings=np.array(batch_embeddings),
                ids=batch_ids
            )

all records >>> 0


Adding data to DB:   0%|          | 0/402 [00:00<?, ?it/s]

Skipping duplicate document: [0a] Title: Rol...
Skipping duplicate document: [0a] Among the ...
Skipping duplicate document: [0b] Expression...
Skipping duplicate document: [0c] Furthermor...
Skipping duplicate document: [0d] Amino-acid...
Skipping duplicate document: [0a] Title: Vir...
Skipping duplicate document: [0a] Title: MER...
Skipping duplicate document: [0a] Defining c...
Skipping duplicate document: [0b] For the vi...
Skipping duplicate document: [1c] Unlike SAR...
Skipping duplicate document: [1d] SARS-CoV a...
Skipping duplicate document: [0a] Title: Chi...
Skipping duplicate document: [1e] Such exace...
Skipping duplicate document: [0a] Title: Vir...
Skipping duplicate document: [0a] A Personal...
Skipping duplicate document: [0b] They were ...
Skipping duplicate document: [0c] Among the ...
Skipping duplicate document: [0d] The sequen...
Skipping duplicate document: [0a] Title: A H...
Skipping duplicate document: [0a] Our reBmBa...
Skipping duplicate document: [0b] First,

Adding data to DB:   0%|          | 1/402 [00:02<13:24,  2.01s/it]

Inserted 2500 records into collection 'ragbench_collection_all_baai_v1'.
Skipping duplicate document: [0a] Title: Chi...
Skipping duplicate document: [0a] Title: Sea...
Skipping duplicate document: [0a] HPIV1 and ...
Skipping duplicate document: [0b] Similar re...
Skipping duplicate document: [0c] According ...
Skipping duplicate document: [0a] Title: Sil...
Skipping duplicate document: [0a] Hepcidin n...
Skipping duplicate document: [0b] Hepcidin b...
Skipping duplicate document: [0c] Because ir...
Skipping duplicate document: [0d] Furthermor...
Skipping duplicate document: [0a] Title: Ide...
Skipping duplicate document: [0a] Moreover, ...
Skipping duplicate document: [0b] When CD40L...
Skipping duplicate document: [0a] Title: Ana...
Skipping duplicate document: [0a] All approa...
Skipping duplicate document: [0b] In total 1...
Skipping duplicate document: [0c] Our drop-o...
Skipping duplicate document: [0d] Table 1 de...
Skipping duplicate document: [0e] No statist...
Skipping duplic

...

Inserted 2500 records into collection 'ragbench_collection_all_baai_v1'.
Skipping duplicate document: [0ew] 5....
Skipping duplicate document: [0fc] 7....
Skipping duplicate document: [0u] 2....
Skipping duplicate document: [0x] 3....
Skipping duplicate document: [0bh] 5....
Skipping duplicate document: [0cb] 8....
Skipping duplicate document: [0ei] 16....
Skipping duplicate document: [0ey] 20....
Skipping duplicate document: [0d] Definition...
Skipping duplicate document: [0be] 4....
Skipping duplicate document: [0a] INTELLECTU...
Skipping duplicate document: [0b] WHEREAS:

...
Skipping duplicate document: [0c] Party A, a...
Skipping duplicate document: [0d] 2....
Skipping duplicate document: [0e] Party B is...
Skipping duplicate document: [0f] Party A ag...
Skipping duplicate document: [0g] NOW THEREF...
Skipping duplicate document: [0h] Grant of L...
Skipping duplicate document: [0i] The licens...
Skipping duplicate document: [0j] 1.1.2   Pa...
Skipping duplicate document: [0k] The 

Adding data to DB:   4%|▍         | 17/402 [01:01<11:57,  1.86s/it]

Inserted 2500 records into collection 'ragbench_collection_all_baai_v1'.
Skipping duplicate document: [0qq] 15.3....
Skipping duplicate document: [0e] 2....
Skipping duplicate document: [0i] 3....
Skipping duplicate document: [0t] 4....
Skipping duplicate document: [0aa] 5....
Skipping duplicate document: [0a] QuickLinks...
Skipping duplicate document: [0b] 1....
Skipping duplicate document: [0c] The Offeri...
Skipping duplicate document: [0d] The Compan...
Skipping duplicate document: [0e] The Compan...
Skipping duplicate document: [0f] 333-135900...
Skipping duplicate document: [0g] Except whe...
Skipping duplicate document: [0h] Except whe...
Skipping duplicate document: [0i] Except whe...
Skipping duplicate document: [0j] "Permitted...
Skipping duplicate document: [0k] "Disclosur...
Skipping duplicate document: [0l] "Blue Sky ...
Skipping duplicate document: [0m] Any refere...
Skipping duplicate document: [0n] Any refere...
Skipping duplicate document: [0o] 2....
Skipping duplicate

Adding data to DB:   4%|▍         | 18/402 [01:02<10:55,  1.71s/it]

Inserted 2500 records into collection 'ragbench_collection_all_baai_v1'.
Skipping duplicate document: [0b] SUCH IDENT...
Skipping duplicate document: [0d] DEFINITION...
Skipping duplicate document: [0l] 2....
Skipping duplicate document: [0q] 3....
Skipping duplicate document: [0x] 4....
Skipping duplicate document: [0av] 8....
Skipping duplicate document: [0cl] 14....
Skipping duplicate document: [0dh] 19....
Skipping duplicate document: [0dk] 20....


In [ ]:
print(f"total records in datastore {datastor.has_entities(collection_name)}")